In [ ]:
!pip install transformers
!pip install datasets
!pip install --upgrade accelerate
!pip install rank-eval
!pip install ranx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.autonotebook import trange
import numpy as np
import json
import random
from torch.utils.data import DataLoader, RandomSampler
from tqdm import tqdm
from ranx import Qrels, Run, evaluate

In [8]:
import os
import gc
from collections import defaultdict
import heapq
from scipy.sparse import csr_matrix
import pandas as pd

In [13]:
from utils import tsv_to_dict_multiple, tsv_to_dict_unqiue, split_embedding_into_chunks

In [27]:
model = None
trainer = None
gc.collect()
torch.cuda.empty_cache()

In [28]:
!nvidia-smi

Sat May  3 02:45:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             70W /  400W |   16041MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Load raw training data

In [6]:
# base-model
# model_name = 'facebook/contriever'
# fine-tuned MS-MARCO
model_name = "facebook/contriever-msmarco"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
passages = tsv_to_dict_unqiue(os.path.join('/content/drive/MyDrive/685/data', "collection.tsv"))
queries = tsv_to_dict_unqiue(os.path.join('/content/drive/MyDrive/685/data', "queries.train.tsv"))

In [5]:
def create_dataset(address, n_negatives=2,cache_dir='hf',sample_count=1000):
    def train_gen():
      with open(address,'r') as f:
          count = 0
          for line in f:
              count += 1
              if sample_count != None and count >= sample_count: return
              data_sample = json.loads(line)
              negatives = []
              for neg_id in random.sample(data_sample['neg']['bm25'],n_negatives): negatives.append(passages[str(neg_id)])
              yield {'query': queries[str(data_sample['qid'])],'positive':passages[str(random.sample(data_sample['pos'],1)[0])],'negatives':negatives}
    return Dataset.from_generator(train_gen, cache_dir = cache_dir)

In [18]:
dataset = create_dataset(os.path.join('/content/drive/MyDrive/685/data','msmarco-hard-negatives-bm25_1k.jsonl'), 2, 'hf',500000)

In [18]:
def create_dataset(df, n_negatives=1,cache_dir='hf',sample_count=1000):
    def train_gen():
      samp_df = df.sample(n=sample_count, random_state=42)
      for index, row in samp_df.iterrows():
            yield {'query': queries[str(int(row[2]))],'positive':passages[str(int(row[3]))],'negatives':passages[str(int(row[4]))]}
    return Dataset.from_generator(train_gen, cache_dir = cache_dir)

In [19]:
df = pd.read_csv(os.path.join('/content/drive/MyDrive/685/data','bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv'),sep='\t',header=None, index_col=None)

In [20]:
dataset = create_dataset(df, 1, 'hf',50)

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
split_dataset = dataset.train_test_split(test_size=0.1)

# There might be a split over
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

### Custom data collator

In [8]:
from typing import Any, Dict, List, Union

class CustomDataCollatorWithPadding(DataCollatorWithPadding):
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        query_texts = []
        pos_texts = []
        neg_texts = []

        for feature in features:
            query_texts.append(feature['query'])
            pos_texts.append(feature['positive'])
            if 'negatives' in feature.keys():
                for neg_text in feature['negatives']: neg_texts.append(neg_text)

        tokenized_query_texts = self.tokenizer(
                query_texts,
                max_length=self.max_length,
                padding=self.padding,
                truncation=True,
                return_tensors="pt",
                add_special_tokens=True)

        tokenized_pos_texts = self.tokenizer(
                pos_texts,
                max_length=self.max_length,
                padding=self.padding,
                truncation=True,
                return_tensors="pt",
                add_special_tokens=True)

        tokenized_texts = {
            'tokenized_queries' : tokenized_query_texts,
            'tokenized_positives' : tokenized_pos_texts,
        }


        if len(neg_texts) > 0:
            tokenized_neg_texts = self.tokenizer(
                  neg_texts,
                  max_length=self.max_length,
                  padding=self.padding,
                  truncation=True,
                  return_tensors="pt",
                  add_special_tokens=True)
            tokenized_texts['tokenized_negatives'] = tokenized_neg_texts

        return tokenized_texts

custom_data_collator = CustomDataCollatorWithPadding(
    tokenizer=tokenizer,
    padding='longest',
    max_length=tokenizer.model_max_length
)

In [9]:
out_train = custom_data_collator([train_dataset[i] for i in range(8)])
print(out_train['tokenized_queries']['input_ids'].shape)
print(out_train['tokenized_positives']['input_ids'].shape)
print(out_train['tokenized_negatives']['input_ids'].shape)

NameError: name 'train_dataset' is not defined

In [13]:
out_eval = custom_data_collator([eval_dataset[i] for i in range(2)])
print(out_eval['tokenized_queries']['input_ids'].shape)
print(out_eval['tokenized_positives']['input_ids'].shape)

torch.Size([2, 10])
torch.Size([2, 73])


### Contrastive loss

In [10]:
#enter your code here
def get_contriever_emb(model_output, attention_mask):
    last_hidden = model_output["last_hidden_state"]
    last_hidden = last_hidden.masked_fill(~attention_mask[..., None].bool(), 0.0)
    emb = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    return emb

def contrastive_loss(query_embs,pos_embs,neg_embs):
    temp = 0.05
    query_embs = query_embs/temp
    similarities_pos = torch.mm(query_embs,torch.transpose(pos_embs,0,1))
    similarities = similarities_pos
    if neg_embs != None:
        similarities_neg = torch.mm(query_embs,torch.transpose(neg_embs,0,1))
        similarities = torch.cat((similarities_pos,similarities_neg),dim=1)
    return F.cross_entropy(similarities,torch.arange(0,query_embs.shape[0]).to('cuda'))




### Models

In [11]:
class Quantize(nn.Module):
    def __init__(self, dim, n_embed, decay=0.99, eps=1e-5):
        super().__init__()
        self.dim = dim
        self.n_embed = n_embed
        self.decay = decay
        self.eps = eps

        embed = torch.randn(dim, n_embed)
        self.register_buffer("embed", embed)
        self.register_buffer("cluster_size", torch.zeros(n_embed))
        self.register_buffer("embed_avg", embed.clone())

    def forward(self, input,training=True):
        flatten = input.reshape(-1, self.dim)
        dist = (
            flatten.pow(2).sum(1, keepdim=True)
            - 2 * flatten @ self.embed
            + self.embed.pow(2).sum(0, keepdim=True)
        )
        _, embed_ind = (-dist).max(1)
        embed_onehot = F.one_hot(embed_ind, self.n_embed).type(flatten.dtype)
        embed_ind = embed_ind.view(*input.shape[:-1])
        quantize = self.embed_code(embed_ind)

        if training:
            embed_onehot_sum = embed_onehot.sum(0)
            embed_sum = flatten.transpose(0, 1) @ embed_onehot

            # If not using distributed training, comment these out
            # dist_fn.all_reduce(embed_onehot_sum)
            # dist_fn.all_reduce(embed_sum)

            self.cluster_size.data.mul_(self.decay).add_(
                embed_onehot_sum, alpha=1 - self.decay
            )
            self.embed_avg.data.mul_(self.decay).add_(embed_sum, alpha=1 - self.decay)

            n = self.cluster_size.sum()
            cluster_size = (self.cluster_size + self.eps) / (n + self.n_embed * self.eps) * n
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(0)
            self.embed.data.copy_(embed_normalized)

            # --- Dead code reinitialization ---
            with torch.no_grad():
                dead_mask = self.cluster_size < 0.1
                num_dead = dead_mask.sum().item()
                if num_dead > 0 and flatten.size(0) > 0:
                    random_indices = torch.randint(0, flatten.size(0), (num_dead,))
                    self.embed[:, dead_mask] = flatten[random_indices].t()
                    self.embed_avg[:, dead_mask] = flatten[random_indices].t()
                    self.cluster_size[dead_mask] = 1.0  # or a small positive value

        diff = (quantize.detach() - input).pow(2).mean()
        quantize = input + (quantize - input).detach()

        return quantize, embed_ind

    def embed_code(self, embed_id):
        return F.embedding(embed_id, self.embed.transpose(0, 1))

In [12]:
class Sparse_Layer(nn.Module):
    def __init__(self,d,D):
        super().__init__()

        self.up_projection = nn.Linear(d,D)
        nn.init.xavier_uniform_(self.up_projection.weight)
        nn.init.zeros_(self.up_projection.bias)

    def forward(self,embeds):
        # print("NaNs before matmul:", torch.isnan(embeds).any())
        S = self.up_projection(embeds)
        # print("NaNs after projection:", torch.isnan(S).any())
        S = torch.log(1 + torch.relu(S))
        # print("NaNs after logrelu:", torch.isnan(S).any())
        S = torch.clamp(S, max=10)
        # print("NaNs after clamp:", torch.isnan(S).any())
        # S = torch.nn.functional.normalize(S, p=2, dim=1,eps=1e-12)
        # print("NaNs after normalize:", torch.isnan(S).any())
        return S

In [28]:
class LSR_VQ(nn.Module):
    def __init__(self, model_name, emb_dim, num_clusters, num_chunks, d, D, use_quantization = True, freeze_contriver=False):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name,device_map='auto')
        if freeze_contriver:
            for param in self.model.parameters():
                param.requires_grad = False
        self.quantizer = Quantize(emb_dim,num_clusters)
        self.sparse_layer = Sparse_Layer(d,D)
        self.num_chunks = num_chunks
        self.use_quantization = use_quantization

    def forward(self, inputs):
        query_embs = self.model(**inputs['tokenized_queries'])
        query_dense = get_contriever_emb(query_embs,inputs['tokenized_queries']['attention_mask'])

        pos_embs = self.model(**inputs['tokenized_positives'])
        pos_dense = get_contriever_emb(pos_embs,inputs['tokenized_positives']['attention_mask'])

        neg_embs = self.model(**inputs['tokenized_negatives'])
        neg_dense = get_contriever_emb(neg_embs,inputs['tokenized_negatives']['attention_mask'])

        if self.use_quantization :

            embeds_to_quantize = torch.cat([query_dense,pos_dense,neg_dense],dim=0)

            embeds = split_embedding_into_chunks(embeds_to_quantize,self.num_chunks)

            quantized_embs_chunks, _ = self.quantizer(embeds,True)

            bsz_k, dim_chunk = quantized_embs_chunks.shape
            bsz = bsz_k // self.num_chunks
            quantized_embs = quantized_embs_chunks.view(bsz, dim_chunk * self.num_chunks)

            query_offset = 0
            pos_offset = query_dense.shape[0]
            neg_offset = query_dense.shape[0] + pos_dense.shape[0]

            query_quantized_embs = quantized_embs[:pos_offset]
            pos_quantized_embs = quantized_embs[pos_offset:neg_offset]
            neg_quantized_embs = quantized_embs[neg_offset:]

        else:
          query_quantized_embs = query_dense
          pos_quantized_embs = pos_dense
          neg_quantized_embs = neg_dense

        query_sparse = self.sparse_layer(query_quantized_embs)
        pos_sparse = self.sparse_layer(pos_quantized_embs)
        neg_sparse = self.sparse_layer(neg_quantized_embs)

        return query_dense,query_sparse,pos_dense,pos_sparse,neg_dense,neg_sparse

    def get_sparse_rep(self,inputs):
        embs = self.model(**inputs)
        dense_embs = get_contriever_emb(embs,inputs['attention_mask'])

        if self.use_quantization:
            embeds = split_embedding_into_chunks(dense_embs,self.num_chunks)

            quantized_embs_chunks, _ = self.quantizer(embeds,False)

            bsz_k, dim_chunk = quantized_embs_chunks.shape
            bsz = bsz_k // self.num_chunks
            quantized_embs = quantized_embs_chunks.view(bsz, dim_chunk * self.num_chunks)
        else:
            quantized_emds = dense_embs

        sparse_rep = self.sparse_layer(quantized_embs)

        return sparse_rep

In [37]:
num_chunks = 1
vocab_size = 65536
cluster_size = 1024

In [29]:
model = LSR_VQ(model_name,768//num_chunks,cluster_size,num_chunks,768,vocab_size)

In [30]:
from safetensors.torch import load_file

In [31]:
# state_dict = load_file("weights/checkpoint-6500/model.safetensors")
state_dict = load_file("/content/drive/MyDrive/685/checkpoints/checkpoint-7072/model.safetensors")
model.load_state_dict(state_dict)

<All keys matched successfully>

### Training

In [26]:
#enter your code here to define trainer
class LSRVQ_Trainer(Trainer):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.query_reg_lambda = 5e-3
        self.passage_reg_lambda = 5e-3

    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):

        query_dense, query_sparse, pos_dense, pos_sparse, neg_dense, neg_sparse = self.model(inputs)
        # print(query_sparse[:4])
        # Constrastive loss
        c_loss = contrastive_loss(query_dense,pos_dense,neg_dense)

        # print('c_loss',c_loss)

        # Query sparsity loss
        query_reg_loss = torch.sum(torch.mean(torch.abs(query_sparse), dim=0) ** 2)

        # print('q_loss',query_reg_loss)
        # count = (query_sparse > 1e-5).sum().item()
        # print(count)

        # Document sparsity loss
        passage_sparse = torch.cat([pos_sparse,neg_sparse], dim=0)
        passage_reg_loss = torch.sum(torch.mean(torch.abs(passage_sparse), dim=0) ** 2)

        # print(passage_sparse[:4])
        # print('p_loss',passage_reg_loss)

        # if self.state.global_step%500 == 0: print(c_loss,query_reg_loss,passage_reg_loss)

        loss = c_loss + self.query_reg_lambda*query_reg_loss + self.passage_reg_lambda*passage_reg_loss

        return (loss,torch.zeros(1)) if return_outputs else loss

    # def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):

    #     query_dense, query_sparse, pos_dense, pos_sparse, neg_dense, neg_sparse = self.model(inputs)
    #     c_loss = contrastive_loss(query_dense,pos_dense,neg_dense)

    #     loss = c_loss

    #     return (loss,torch.zeros(1)) if return_outputs else loss



In [27]:
training_args = TrainingArguments(
    output_dir="./weights",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    eval_steps=500,
    eval_strategy="steps",
    save_total_limit=50,
    save_steps=500,
    load_best_model_at_end=True,
    greater_is_better=False,
    learning_rate=1e-4,
    lr_scheduler_type = "linear",
    fp16=True,
    report_to='none',
    metric_for_best_model="eval_loss",
    remove_unused_columns=False,
)

In [28]:
trainer = LSRVQ_Trainer(
        model = model,
        args = training_args,
        data_collator = custom_data_collator,
        train_dataset = train_dataset,
        eval_dataset = eval_dataset)

trainer.can_return_loss=True

def custom_eval_dataloader(eval_set):
    return DataLoader(
    eval_set,
    batch_size=training_args.eval_batch_size,
    sampler=RandomSampler(eval_dataset),
    collate_fn=trainer.data_collator,
)

trainer.get_eval_dataloader = custom_eval_dataloader

In [29]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.293400,0.252728
1000,0.284900,0.264018
1500,0.273700,0.254062
2000,0.265800,0.248452
2500,0.261500,0.233299
3000,0.260300,0.222684
3500,0.241100,0.218569
4000,0.229800,0.204919
4500,0.220200,0.192291
5000,0.204700,0.190551


KeyboardInterrupt: 

### Inference

In [24]:
queries_dev = tsv_to_dict_unqiue(os.path.join('/content/drive/MyDrive/685/data', "queries.dev.small.tsv"))
qrels_dev = tsv_to_dict_multiple(os.path.join('/content/drive/MyDrive/685/data', "qrels.dev.small.tsv"), keys = [0, 2])

with open('/content/drive/MyDrive/685/embeddings/dev/passage_ids.json') as f:
    inference_passage_ids = json.load(f)

In [32]:
from datasets import load_dataset

def encode_texts(model, tokenizer, text_ds, batch_size, verbose=False):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    text_ids = [i['_id'] for i in text_ds]
    dataloader = DataLoader(text_ds, batch_size=batch_size, shuffle=False)
    allemb = []

    model.to(device)
    model.eval()

    with torch.no_grad():
        for batch in tqdm(dataloader):
            prepare_batch = [batch['text'][i] for i in range(len(batch['text']))]
            tokenized_texts = tokenizer.batch_encode_plus(
                prepare_batch,
                max_length=tokenizer.model_max_length,
                padding=True,
                truncation=True,
                return_tensors="pt",
                add_special_tokens=True,
            ).to(device)
            if verbose:
                print("Batch size:", len(batch['text']))
                print(f"Tokenized texts shape: {tokenized_texts['input_ids'].shape}")
                print(f"Decode text sample {tokenizer.decode(tokenized_texts['input_ids'][0])}")
            emb = model.get_sparse_rep(tokenized_texts)
            # allemb.append(emb.cpu())
            allemb.append(emb)
    allemb = torch.cat(allemb, dim=0)
    # allemb = allemb.cpu().numpy()
    return allemb, text_ids

In [33]:
corpus_ds = Dataset.from_dict({'_id':inference_passage_ids,'text':[passages[_id] for _id in inference_passage_ids]})
queries_ds = Dataset.from_dict({'_id':list(queries_dev.keys()),'text':list(queries_dev.values())})

### Top-k Sparsification

In [34]:
passage_topk = 1024

passage_sparse_embeddings, passage_ids = encode_texts(model, tokenizer, corpus_ds, batch_size=128)


100%|██████████| 59/59 [01:15<00:00,  1.28s/it]


In [1]:
len(passage_ids)

NameError: name 'passage_ids' is not defined

In [35]:
# Apply top-k sparsification
values, indices = torch.topk(passage_sparse_embeddings.abs(), k=passage_topk, dim=1)
sparse_embeddings = torch.zeros_like(passage_sparse_embeddings)
sparse_embeddings.scatter_(1, indices, passage_sparse_embeddings.gather(1, indices))

passage_sparse_embeddings = sparse_embeddings.detach().cpu().numpy()

### Inverted Index

In [44]:
rows, cols, data = [], [], []

min_weight = 1e-5
for j, doc_id in enumerate(passage_ids):
    doc_embedding = passage_sparse_embeddings[j]
    # Get non-zero indices and their values
    nonzero_indices = np.nonzero(doc_embedding)[0]  # Ensure indices are integers
    for idx in nonzero_indices:
        weight = doc_embedding[idx]
        if weight > min_weight:
          rows.append(j)
          cols.append(idx)
          data.append(float(weight))

optimized_index = csr_matrix((data, (rows, cols)), shape = (len(passage_ids), vocab_size), dtype = np.float32)

### Search

In [46]:
query_sparse_embeddings, query_ids = encode_texts(model, tokenizer, queries_ds, batch_size=128)

100%|██████████| 55/55 [00:09<00:00,  5.75it/s]


In [47]:
query_topk = 256
values, indices = torch.topk(query_sparse_embeddings.abs(), k=query_topk, dim=1)
sparse_embeddings = torch.zeros_like(query_sparse_embeddings)
sparse_embeddings.scatter_(1, indices, query_sparse_embeddings.gather(1, indices))
sparse_embeddings[sparse_embeddings.abs() < min_weight] = 0
query_sparse_embeddings = sparse_embeddings.detach().cpu().numpy()

In [48]:
batch_size = 512
all_results = {}
for start in tqdm(range(0, len(queries_ds), batch_size), desc = "Evaluating queries (csr)..."):
    end = min(start + batch_size, len(queries_ds))
    query_chunk = query_sparse_embeddings[start:end]
    scores_chunk = query_chunk @ optimized_index.T  # shape: [batch_size, num_passages]

    topk_idx = np.argpartition(-scores_chunk, 1000, axis = 1)[:, :1000]
    for i in range(scores_chunk.shape[0]):
        passage_scores = scores_chunk[i]
        top_i = topk_idx[i]
        top_indices = top_i[np.argsort(-passage_scores[top_i])]

        # Ensure self.passage_ids is referring to the correct mode (train/dev)
        all_results[query_ids[start + i]] = [(passage_ids[t], passage_scores[t]) for t in top_indices]

Evaluating queries (csr)...: 100%|██████████| 14/14 [00:06<00:00,  2.06it/s]


### Metrics

In [49]:
run_dict = {}
for qid, results in all_results.items():
    run_dict[qid] = {
        str(passage_id): float(score)
        for passage_id, score in results
    }
run = Run(run_dict)

qrels_dict = {
    qid: {str(passage_id): 1 for passage_id in qrels_dev[qid]}
    for qid in all_results.keys()
}
qrels = Qrels(qrels_dict)

# Evaluate using rankx
metrics = ["ndcg@10", "ndcg@100", "ndcg@1000", "recall@10", "recall@100", "recall@1000", "mrr@10"]
results = evaluate(qrels, run, metrics)

m = (results["mrr@10"],
    {
        '10': results["ndcg@10"],
        '100': results["ndcg@100"],
        '1000': results["ndcg@1000"]
    },
    {
        '10': results["recall@10"],
        '100': results["recall@100"],
        '1000': results["recall@1000"]
    }
)

In [42]:
m

(np.float64(0.0006737503979624323),
 {'10': np.float64(0.000932891863126572),
  '100': np.float64(0.003118251768003201),
  '1000': np.float64(0.017056010516315546)},
 {'10': np.float64(0.0017908309455587394),
  '100': np.float64(0.0136341929321872),
  '1000': np.float64(0.13477793696275073)})

In [54]:
m

(np.float64(0.01013309000773184),
 {'10': np.float64(0.012834261814959424),
  '100': np.float64(0.0446723789549938),
  '1000': np.float64(0.05988806483628583)},
 {'10': np.float64(0.02255253104106972),
  '100': np.float64(0.18547039159503342),
  '1000': np.float64(0.31166427889207265)})

In [118]:
m

(np.float64(0.0252201300768636),
 {'10': np.float64(0.030283006047204145),
  '100': np.float64(0.0645143676048759),
  '1000': np.float64(0.09501735668649862)},
 {'10': np.float64(0.04844794651384909),
  '100': np.float64(0.23106494746895892),
  '1000': np.float64(0.4693767908309456)})

In [59]:
m

(np.float64(0.06048903897757765),
 {'10': np.float64(0.0747259984359336),
  '100': np.float64(0.11962298762407109),
  '1000': np.float64(0.16165079743097477)},
 {'10': np.float64(0.126432664756447),
  '100': np.float64(0.3510028653295129),
  '1000': np.float64(0.69481852913085)})

In [53]:
m

(np.float64(0.013082048483194615),
 {'10': np.float64(0.02108233537348365),
  '100': np.float64(0.04920603383509248),
  '1000': np.float64(0.0621292154232465)},
 {'10': np.float64(0.04869866284622732),
  '100': np.float64(0.17944126074498568),
  '1000': np.float64(0.2898758357211079)})

In [55]:
!cp -R './weights/checkpoint-7072' '/content/drive/MyDrive/685/checkpoints'

In [54]:
for i in range(len(query_sparse_embeddings)):
  print(np.where(query_sparse_embeddings[i] > 0)[0].shape)

Streaming output truncated to the last 5000 lines.
(4,)
(0,)
(6,)
(0,)
(0,)
(2,)
(3,)
(20,)
(0,)
(0,)
(2,)
(1,)
(1,)
(0,)
(0,)
(0,)
(0,)
(12,)
(0,)
(0,)
(2,)
(0,)
(0,)
(0,)
(0,)
(4,)
(0,)
(0,)
(1,)
(0,)
(2,)
(0,)
(2,)
(0,)
(9,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(2,)
(19,)
(5,)
(0,)
(7,)
(1,)
(0,)
(3,)
(0,)
(0,)
(2,)
(3,)
(0,)
(12,)
(0,)
(0,)
(0,)
(0,)
(0,)
(1,)
(256,)
(0,)
(0,)
(0,)
(0,)
(12,)
(7,)
(3,)
(25,)
(14,)
(0,)
(0,)
(0,)
(5,)
(2,)
(0,)
(0,)
(7,)
(25,)
(0,)
(1,)
(1,)
(1,)
(3,)
(1,)
(1,)
(1,)
(1,)
(0,)
(0,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)
(3,)
(0,)
(0,)
(0,)
(0,)
(0,)
(65,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(20,)
(0,)
(1,)
(15,)
(0,)
(0,)
(0,)
(0,)
(5,)
(15,)
(3,)
(256,)
(7,)
(0,)
(0,)
(0,)
(0,)
(2,)
(181,)
(256,)
(4,)
(2,)
(0,)
(4,)
(0,)
(0,)
(0,)
(7,)
(26,)
(0,)
(16,)
(0,)
(0,)
(0,)
(0,)
(15,)
(0,)
(65,)
(1,)
(0,)
(0,)
(8,)
(7,)
(0,)
(1,)
(256,)
(4,)
(208,)
(3,)
(0,)
(0,)
(0,)
(25,)
(0,)
(0,)
(0,)
(0,)
(0,)
(3,)
(0,)
(5,)
(0,)
(0,)
(0,)
(10,)
(0,)
(4,)
(1,)
(0,)
(7,)
(2,)
(0,)

In [55]:
for i in range(len(passage_sparse_embeddings)):
  print(np.where(passage_sparse_embeddings[i] > 0)[0].shape)

Streaming output truncated to the last 5000 lines.
(1,)
(14,)
(1,)
(0,)
(3,)
(3,)
(4,)
(0,)
(14,)
(2,)
(1,)
(1,)
(4,)
(3,)
(3,)
(1,)
(5,)
(5,)
(30,)
(5,)
(0,)
(2,)
(0,)
(3,)
(4,)
(0,)
(0,)
(0,)
(4,)
(9,)
(12,)
(4,)
(3,)
(4,)
(8,)
(1,)
(1,)
(1,)
(6,)
(3,)
(1,)
(4,)
(1,)
(8,)
(1,)
(0,)
(7,)
(0,)
(1,)
(12,)
(3,)
(6,)
(11,)
(3,)
(0,)
(0,)
(2,)
(0,)
(0,)
(2,)
(2,)
(1,)
(4,)
(2,)
(3,)
(0,)
(3,)
(3,)
(1,)
(23,)
(14,)
(7,)
(1,)
(3,)
(2,)
(6,)
(4,)
(2,)
(10,)
(2,)
(9,)
(0,)
(1,)
(5,)
(6,)
(6,)
(2,)
(4,)
(2,)
(12,)
(1,)
(2,)
(2,)
(1,)
(1,)
(5,)
(6,)
(3,)
(14,)
(9,)
(12,)
(0,)
(11,)
(1,)
(0,)
(5,)
(0,)
(1,)
(0,)
(1,)
(5,)
(2,)
(5,)
(0,)
(4,)
(5,)
(0,)
(15,)
(2,)
(2,)
(6,)
(11,)
(0,)
(6,)
(7,)
(0,)
(2,)
(8,)
(3,)
(5,)
(15,)
(1,)
(1,)
(4,)
(0,)
(4,)
(2,)
(8,)
(6,)
(2,)
(3,)
(1,)
(6,)
(0,)
(0,)
(7,)
(3,)
(4,)
(15,)
(1,)
(0,)
(1,)
(1,)
(0,)
(4,)
(0,)
(8,)
(5,)
(6,)
(12,)
(3,)
(4,)
(7,)
(1,)
(1,)
(33,)
(0,)
(2,)
(0,)
(0,)
(6,)
(0,)
(1,)
(1,)
(0,)
(2,)
(7,)
(0,)
(1,)
(5,)
(1,)
(2,)
(20,)
(0,)
(3,)
(0,)